In [ ]:
import gradio as gr
import cv2
import numpy as np
import matplotlib.pyplot as plt
from deepface import DeepFace
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

def extract_skin_mask(image):
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_image)
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = []
            for idx in range(234, 454):
                x = int(face_landmarks.landmark[idx].x * image.shape[1])
                y = int(face_landmarks.landmark[idx].y * image.shape[0])
                landmarks.append([x, y])
            hull = cv2.convexHull(np.array(landmarks))
            cv2.drawContours(mask, [hull], -1, 255, -1)
    mask = cv2.erode(mask, np.ones((5, 5), np.uint8), iterations=1)
    return mask, results.multi_face_landmarks

def draw_face_landmarks(image, face_landmarks):
    annotated_image = image.copy()
    if face_landmarks:
        for landmarks in face_landmarks:
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_tesselation_style())
            
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_contours_style())
    return annotated_image

def plot_emotions(emotions: dict):
    labels = list(emotions.keys())
    values = list(emotions.values())
    fig, ax = plt.subplots(figsize=(6, 4))
    bars = ax.bar(labels, values, color='skyblue')
    ax.set_ylim(0, 100)
    ax.set_ylabel('Percentage (%)')
    ax.set_title('Emotion Distribution')
    ax.bar_label(bars, fmt='%.1f%%')
    plt.xticks(rotation=45)
    plt.tight_layout()
    return fig

def analyze_health(image):
    try:
        image = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_RGB2BGR)
        
        result = DeepFace.analyze(img_path=image, actions=['emotion', 'age'], enforce_detection=False)
        emotions = result[0]['emotion']
        age = result[0]['age']
        emotion_fig = plot_emotions(emotions)

        mask, face_landmarks = extract_skin_mask(image)
        if np.sum(mask) == 0:
            raise ValueError("No face detected")
        
        annotated_image = draw_face_landmarks(image, face_landmarks)
        annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)

        skin_rgb = image[mask > 0]
        r_mean = np.mean(skin_rgb[:, 2]) if skin_rgb.size > 0 else 0
        
        lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
        lab = lab.astype(np.float32)
        lab[:,:,0] = lab[:,:,0] * (100/255) # L: 0-100
        lab[:,:,1] = lab[:,:,1] - 128       # a: -128 to 127
        lab[:,:,2] = lab[:,:,2] - 128       # b: -128 to 127
        skin_lab = lab[mask > 0]
        
        l_mean = np.mean(skin_lab[:, 0]) if skin_lab.size > 0 else 0
        a_mean = np.mean(skin_lab[:, 1]) if skin_lab.size > 0 else 0
        b_mean = np.mean(skin_lab[:, 2]) if skin_lab.size > 0 else 0
        
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        contrast = gray.std()  # 0-255
        
        hydration = cv2.Laplacian(gray, cv2.CV_64F).var()

        health_report = (
            f"Basic Metrics:\n"
            f"• Estimated Age: {int(age)} years\n"
            f"• Skin Redness (RGB): {r_mean:.1f}/255\n"
            f"   - {'High (possible irritation)' if r_mean > 180 else 'Normal' if r_mean > 100 else 'Low'}\n\n"
            
            f"Medical Skin Analysis (LAB):\n"
            f"• Brightness (L): {l_mean:.1f}/100\n"
            f"   - {'Light' if l_mean > 70 else 'Normal' if l_mean > 50 else 'Dark'}\n"
            f"• Red-Green (a): {a_mean:.1f} (Norm: 5-20)\n"
            f"   - {'Redness' if a_mean > 20 else 'Normal' if a_mean > 5 else 'Pale'}\n"
            f"• Blue-Yellow (b): {b_mean:.1f} (Norm: 10-25)\n"
            f"   - {'Yellow tint' if b_mean > 30 else 'Normal' if b_mean > 10 else 'Cool'}\n\n"
            
            f"Skin Quality:\n"
            f"• Contrast: {contrast:.1f}/255 (Norm: 40-70)\n"
            f"   - {'Low (fatigue)' if contrast < 35 else 'Normal' if contrast <= 70 else 'High (harsh light)'}\n"
            f"• Hydration: {hydration:.1f} (Norm: 100-300)\n"
            f"   - {'Dry' if hydration < 100 else 'Normal' if hydration <= 300 else 'Well-hydrated'}\n\n"
            
            f"Quick Tips:\n"
            f"- Redness > 180? Check for irritation\n"
            f"- Hydration < 100? Drink more water\n"
            f"- Contrast < 35? Try better lighting"
        )

        return annotated_image, emotion_fig, health_report

    except Exception as e:
        print("Analysis error:", str(e))
        return None, None, f"Error: {str(e)}\n📸 Please upload a clear front-facing photo without filters."

iface = gr.Interface(
    fn=analyze_health,
    inputs=gr.Image(type="numpy", label="Upload Face Photo", height=400),
    outputs=[
        gr.Image(label="Face Landmarks", type="numpy", height=400),
        gr.Plot(label="Emotion Analysis"),
        gr.Textbox(label="Health Report", lines=22)
    ],
    title="Face Health Analyzer",
    description="Get detailed skin and emotion analysis with clear interpretations!",
    allow_flagging="never"
)

if __name__ == "__main__":
    iface.launch()

I0000 00:00:1753081323.492763 60367568 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1753081323.495292 60572273 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753081323.500149 60572271 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Action: age: 100%|██████████| 2/2 [00:00<00:00, 22.71it/s]
